In [ ]:
# cek ip untuk mengatasi download limit (restart session jika limit, ip akan berubah)
!curl ipecho.net/plain

# **Lib**

In [ ]:
!pip install mutagen
!pip install alive-progress

In [ ]:
import requests
import os
import re
from pathlib import Path
import time
import subprocess
import json
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from mutagen.mp4 import MP4
from tqdm.notebook import tqdm

# **Function Download File dari PixelDrain**

> optional, bisa upload file langsung atau download dari file hosting lainnya



In [ ]:
def download_pxd(file_id, output_dir="", retries=5, delay=5):
    for attempt in range(retries):
        try:
            info_url = f"https://pixeldrain.com/api/file/{file_id}/info"
            response = requests.get(info_url)

            if response.status_code == 200:
                file_info = response.json()
                file_name = file_info['name']
                file_path = f"{output_dir}/{file_name}"

                file_url = f"https://pixeldrain.com/api/file/{file_id}"
                file_response = requests.get(file_url, stream=True)
                total_size = int(file_response.headers.get('content-length', 0))

                if file_response.status_code == 200:
                    with open(file_path, 'wb') as file, tqdm(
                        desc=f"donlot {file_name}",
                        total=total_size,
                        unit='B',
                        unit_scale=True,
                        unit_divisor=1024,
                    ) as progress_bar:
                        for data in file_response.iter_content(1024):
                            if not data:
                                break
                            file.write(data)
                            progress_bar.update(len(data))
                    # print(f"Donlot berhasil : {file_name}")
                    return file_path
                else:
                    print(f"Percobaan {attempt + 1}: gagal download {file_name}.")
            else:
                print(f"Percobaan {attempt + 1}: gagal dapat info {file_name}.")

        except Exception as e:
            print(f"Percobaan {attempt + 1}: Exception occurred: {e}")

        # tunggu sebentar
        time.sleep(delay)

    print("Failed to download the file after multiple attempts.")
    return "failed"

# **Function Upload File**

### Pixeldrain



In [ ]:
apikey_pxd = '' #api key pixeldrain
# Fungsi untuk mengunggah file ke PixelDrain menggunakan curl dengan PUT
def upload_to_pixeldrain_with_curl(local_file_path):
    upload_url = 'https://pixeldrain.com/api/file/'

    # Perintah curl untuk mengunggah file
    curl_command = [
        'curl', '-T', local_file_path,
        '-u', f':{apikey_pxd}',  # API Key untuk autentikasi
        upload_url
    ]

    # Jalankan perintah curl
    print(f'Upload {local_file_path} to Pixeldrain...')
    result = subprocess.run(curl_command, capture_output=True, text=True)
    if result.returncode == 0:
        try:
            response_data = json.loads(result.stdout)
            if 'id' in response_data:
                download_link = f"https://pixeldrain.com/u/{response_data['id']}"
                print(f"(Pixeldrain) {local_file_path}: {download_link}")
                return download_link
            else:
                print("Upload successful, but no download link found in response.")
                return None
        except json.JSONDecodeError:
            print("Upload successful, but failed to parse JSON response.")
            return None
    else:
        print(f"Upload failed: {result.stderr}")
        return None

### KrakenFiles

In [ ]:
apikey_kfs = '' #api key krakenfiles
# get server token
def get_server_access_token():
    response = requests.get('https://krakenfiles.com/api/server/available')
    response.raise_for_status()
    data = response.json()
    return data
    # return data['serverAccessToken']

def upload_to_krakenfiles_with_curl(local_file_path):
    server_access_token = get_server_access_token()
    upload_url = server_access_token["data"]["url"]

    # curl
    curl_command = [
        'curl', '-X', 'POST', upload_url,
        '-H', f'X-AUTH-TOKEN: {apikey_kfs}',
        '-F', f'serverAccessToken={server_access_token["data"]["serverAccessToken"]}',
        '-F', f'file=@{local_file_path}'
        # '-F', f'title={title}',
        # '-F', f'folderId={folderId}'
    ]

    print(f'Upload {local_file_path} to Krakenfiles...')
    result = subprocess.run(curl_command, capture_output=True, text=True)
    if result.returncode == 0:
        try:
            response_data = json.loads(result.stdout)
            if 'data' in response_data:
                download_link = response_data['data']['url']
                print(f"(Krakenfiles) {local_file_path}: {download_link}")
                return download_link
            else:
                print("Upload successful, but no download link found in response.")
                return None
        except json.JSONDecodeError:
            print("Upload successful, but failed to parse JSON response.")
            return None
    else:
        print(f"Upload failed: {result.stderr}")
        return None

### HexUpload

In [ ]:
apikey_hex = '' #api key hexupload
# get session dan url upload
def get_hexupload_server_info():
    response = requests.get(f'https://hexload.com/api/upload/server?key={apikey_hex}')
    response.raise_for_status()
    data = response.json()
    return data['sess_id'], data['result']
    # return data

# uplot
def upload_to_hexupload_with_curl(local_file_path):
    sess_id, upload_url = get_hexupload_server_info()

    # curl
    curl_command = [
        'curl', '-X', 'POST', upload_url,
        '-F', f'sess_id={sess_id}',
        '-F', f'file=@{local_file_path}'
    ]

    # Jalankan perintah curl
    print(f"Upload {local_file_path} to Hexfiles...")
    result = subprocess.run(curl_command, capture_output=True, text=True)
    if result.returncode == 0:
        try:
            response_data = json.loads(result.stdout)
            if response_data[0]['file_status'] == 'OK':
                download_link = f"https://hexload.com/{response_data[0]['file_code']}"
                print(f"(Hexupload) {local_file_path}: {download_link}")
                return download_link
            else:
                print("Upload to Hexupload successful, but no download link found in response.")
                return None
        except json.JSONDecodeError:
            print("Upload to Hexupload successful, but failed to parse JSON response.")
            return None
    else:
        print(f"Upload to Hexupload failed: {result.stderr}")
        return None

# Function Persiapan

In [ ]:
# get size
def get_file_size(file_path):
    size_bytes = os.path.getsize(file_path)
    size_mb = size_bytes / (1024 * 1024)
    return f"{size_mb:.2f} MB"

# rename prefix
def rename_file(local_filename):
    new_filename = local_filename.replace(old_prefix, 'new-prefix') # ganti prefix file yang didownload
    os.rename(local_filename, new_filename)
    return new_filename

# set metadata title
def set_file_title(file_path, title):
    video = MP4(file_path)
    video["\xa9nam"] = title  # Mengatur meta title
    video.save()

# get file name dan resolusi
def file_ingfo(local_file_path):
    file_name = Path(local_file_path).name
    match = re.search(r'(\d{3,4}p)', file_name)
    if match:
        reso = match.group(1)
        return reso
    else:
        kosong = ""
        return kosong

# Fungsi untuk donlot, rename dan edit meta title
def download_and_prepare_file(file_ids):
    # print(f"Downloading {file_ids}...")
    file_path = download_pxd(file_ids,output_directory) # download dari pixeldrain
    new_file_path = rename_file(file_path)
    set_file_title(new_file_path, Path(new_file_path).name) # set meta title
    return new_file_path

def retry_upload(upload_function, local_file_path, delay=5):
    link = upload_function(local_file_path)
    while not link:
        print(f"Upload Ulang {local_file_path}...")
        time.sleep(delay)  # Delay sebelum mencoba lagi
        link = upload_function(local_file_path)
    return link

# Fungsi upload
def upload_file(local_file_path):
    file_size = get_file_size(local_file_path)
    reso = file_ingfo(local_file_path)

    pixeldrain_link = retry_upload(upload_to_pixeldrain_with_curl, local_file_path)
    krakenfiles_link = retry_upload(upload_to_krakenfiles_with_curl, local_file_path)
    hexupload_link = retry_upload(upload_to_hexupload_with_curl, local_file_path)

    return f'{Path(local_file_path).name}: [[{reso} ({file_size})], [Pixeldrain,{pixeldrain_link}],[KrakenFiles,{krakenfiles_link}],[HexUpload,{hexupload_link}]]'

# **Download**

In [ ]:
# Directory to save downloaded files
output_directory = "/content/Folder"  #folder output
os.makedirs(output_directory, exist_ok=True)

# List of file IDs to download
file_ids = ["nHoSSsRR", "MHCYmQQD", "HEfXxdZe"]

old_prefix = "Otakudesu.io"  #PREFIX FILE DOWNLOAD SEBELUMNYA

with ThreadPoolExecutor() as executor:
    prepared_files = list(executor.map(download_and_prepare_file, file_ids))

# **UPLOAD**

In [ ]:
with ThreadPoolExecutor() as executor:
    download_links = list(executor.map(upload_file, prepared_files))

# **Link Download**

In [ ]:
for item in download_links:
    print(item)
print()